In [46]:
import pandas as pd 
import json
import sentifish
from sentifish import Sentiment
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

In [47]:
with open('Data.jl') as f:
    lines = f.read().splitlines()

In [48]:
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [49]:
print(df_inter.columns)

Index(['json_element'], dtype='object')


In [50]:

df_inter['json_element'].apply(json.loads)

0       {'appId': 'com.casualgamefactory.wordapart', '...
1       {'appId': 'com.noodlecake.typeshift', 'appName...
2       {'appId': 'com.q42.ohno', 'appName': '0h n0', ...
3       {'appId': 'com.finalgamestudio.aurora.hex.jigs...
4       {'appId': 'za.co.twinc.a9letterjumble', 'appNa...
                              ...                        
4995    {'appId': 'org.iggymedia.periodtracker', 'appN...
4996    {'appId': 'com.periodapp.period', 'appName': '...
4997    {'appId': 'com.bm.android.thermometer', 'appNa...
4998    {'appId': 'com.kiroglue.beingdad', 'appName': ...
4999    {'appId': 'com.go.flo', 'appName': 'Period Tra...
Name: json_element, Length: 5000, dtype: object

In [51]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

In [52]:
#'find accounts on the device','modify your contacts','read phone status and identity',
Dangerous=['directly call phone numbers','read call log','read phone status and identity',
          'read sensitive log data','retrieve running apps','delete all app cache data']
appList=[]
temp=[]
permissionList=[]


In [53]:
for i in range(len(df_final)):#from .jl jsonlines 
    #print(df_final['appName'][i])
    temp=df_final['allPermissions'][i]
    #print(temp)
    for j in range(len(temp)):
        #print(temp[j]['permission'],j)
        permissionList.append(temp[j]['permission'])
    if Dangerous[0] in permissionList and Dangerous[1] in permissionList and Dangerous[2] in permissionList and Dangerous[3] in permissionList and Dangerous[4] in permissionList:
        appList.append(df_final['appId'][i])
        permissionList=[]
    else:
        permissionList=[]

In [54]:
#avoid duplicates elements
new_appList = []
for elem in appList:
    if elem not in new_appList:
        new_appList.append(elem) 

In [55]:
print(len(new_appList))
print(new_appList)
#Apps List that statisfy the above condintion

3
['ru.agc.acontactnexttrial', 'net.soti.mobicontrol.androidwork', 'net.dinglisch.android.taskerm']


In [73]:

for app_id in new_appList:
    getCategory = app(
    app_id,
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
    )
    print(getCategory['genre']+"-"+getCategory['title'])

Communication-Phone and Contacts - AGContacts, Lite edition
Business-MobiControl | Android Enterprise
Tools-Tasker


In [77]:
for app_id in new_appList:
    result = reviews_all(
    app_id,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with=5 # defaults to None(means all score)
    )
    reviewList=[]
    for i in range(len(result)):
        reviewList.append(result[i]['content'])
    countPositive=0
    countNegative=0
    for t in range(5):
        obj=Sentiment(reviewList[t])
        polarity = obj.analyze( )
        if obj.isPositive():
            countPositive+=1
        else:
            countNegative+=1
    print(countPositive)
    print(countNegative)
#code for scrape the review of that app

IndexError: tuple index out of range

In [69]:
print(countPositive)
print(countNegative)

5
1


0
0
